<a href="https://www.kaggle.com/code/emigiupponi/usdt-ranking-prices-in-argentina?scriptVersionId=146176871" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Ranking of the final prices -including commissions- of the principal brokers or exchanges of interest in pesos in Argentina

In [1]:
import requests
from datetime import date, datetime, timedelta
import pandas as pd

In [2]:
response = requests.get('https://criptoya.com/api/usdt/ars')
print(response)

<Response [200]>


In [3]:
json = response.json()
df = pd.DataFrame(json)

In [4]:
# Trasformación y cálculos intermedios
df_transposed = df.T
df_transposed.columns = ['Buying', 'Buying (final)', 'Selling', 'Selling (final)', 'Timestamp']
df_transposed['Spread'] = df_transposed["Buying (final)"] - df_transposed["Selling (final)"]
df_transposed['Average dayly price'] = (df_transposed['Buying (final)'] + df_transposed['Selling (final)']) / 2

In [5]:
# Calcular el average de cada columna y agregarlo como una nueva fila
average = df_transposed.mean().to_frame().T
average.index = ['average']
df_transposed = pd.concat([df_transposed, average])

In [6]:
df_transposed = df_transposed .sort_values('Spread', ascending=False)

In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [8]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from datetime import date

# Crear subplots y definir el tamaño de la cuadrícula del gráfico
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03, subplot_titles=('Prices', 'Spread'), 
               row_width=[0.2, 0.7])

# Obtener la fecha actual
current_date = date.today().strftime("%d %b %Y")

# Actualizar el diseño del gráfico
fig.update_layout(
    title=f'<b>USDT prices and spread in ARS for {current_date}<b>',
    title_font_color="#AA322F",
    scene=dict(
        xaxis_title='Date',
    ),
    margin=dict(l=50, r=50, b=50, t=50),
    font=dict(family='Segoe UI', size=11, color='#404040'), # Ajusta la fuente aquí
    paper_bgcolor='white', # Cambia el fondo del gráfico a blanco para un aspecto más formal
)

# Agregar nombres a los ejes Y
fig.update_yaxes(title_text='ARS', row=1, col=1)
fig.update_yaxes(title_text='ARS', row=2, col=1)

# Agregar trazas para 'Compra' y 'Venta' en tonos azules y grises
fig.add_trace(
    go.Scatter(x=df_transposed.index, y=df_transposed['Selling (final)'], name='Venta', line=dict(color='#6CADE1')), # Azul
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df_transposed.index, y=df_transposed['Buying (final)'], name='Compra', line=dict(color='#1C598A')), # Azul claro
    row=1, col=1
)

# Agregar trazas para 'Spread %' en gris y establecer el color rojo para 'average'
colors = ['gray' if exchange != 'average' else '#AA322F' for exchange in df_transposed.index]
fig.add_trace(
    go.Bar(x=df_transposed.index, y=df_transposed['Spread'], name='Spread %', marker_color=colors),
    row=2, col=1
)

# Ocultar el rango deslizante de la gráfica OHLC
fig.update(layout_xaxis_rangeslider_visible=False)

# Mostrar el gráfico
fig.show()

# Times series data

Guardar los datos de hoy en un csv local

In [9]:
# crear el archivo csv con la fecha de ejecución del día a partri de la consulta api de arriba
from datetime import date

# Obtén la fecha actual
today = date.today()

# Convierte la fecha en una cadena en el formato deseado (por ejemplo, '2023-09-13')
date_str = today.strftime("%Y-%m-%d")

# Guarda el DataFrame con el nombre de la fecha
df_transposed.to_csv(f'{date_str}_data.csv', index=True)

Expotar a s3

In [10]:
# expotar el archivo a s3
import boto3
from botocore.exceptions import NoCredentialsError
from kaggle_secrets import UserSecretsClient

# Obtén las claves de acceso a AWS desde los secretos de Kaggle
user_secrets = UserSecretsClient()
aws_access_key_id = user_secrets.get_secret("AWSAccessKeyId")
aws_secret_access_key = user_secrets.get_secret("AWSSecretKey")

# Nombre del archivo y del bucket
file_name = f'{date_str}_data.csv'
bucket_name = "usdt-prices-arg"

# Crea un cliente de S3
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# Intenta crear el bucket
try:
    s3.create_bucket(Bucket=bucket_name)
except Exception as e:
    print(f'Error al crear el bucket: {e}')

# Sube el archivo al bucket de S3
try:
    s3.upload_file(file_name, bucket_name, file_name)
    print(f'{file_name} subido exitosamente a {bucket_name}')
except FileNotFoundError:
    print(f'El archivo {file_name} no se encontró')
except NoCredentialsError:
    print('No se encontraron credenciales de AWS')
except Exception as e:
    print(f'Error al subir el archivo a S3: {e}')

2023-10-11_data.csv subido exitosamente a usdt-prices-arg


Importar desde s3

In [11]:
# Listar los archivos en el bucket de S3:

# Nombre del bucket
bucket_name = "usdt-prices-arg"

# Crea un cliente de S3
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# Lista todos los objetos en el bucket
objects = s3.list_objects(Bucket=bucket_name)

# Extrae los nombres de archivo de la lista de objetos
file_names = [obj['Key'] for obj in objects['Contents']]

In [12]:
# Descargar los archivos desde S3

# Crear un DataFrame vacío para almacenar los datos
combined_df = pd.DataFrame()

# Recorre los nombres de archivo y descárgalos desde S3
for file_name in file_names:
    # Descarga el archivo desde S3
    s3.download_file(bucket_name, file_name, file_name)
    
    # Lee el archivo en un DataFrame
    df = pd.read_csv(file_name)
    
    # Combina el DataFrame descargado con el DataFrame principal
    combined_df = pd.concat([combined_df, df], ignore_index=True)

In [13]:
# Función para convertir timestamp a fecha
def timestamp_to_date(timestamp):
    return pd.to_datetime(timestamp, unit='s').strftime("%d %b %Y")

# Aplica la función a la columna "Timestamp" y crea una nueva columna "Fecha"
combined_df['Fecha'] = combined_df['Timestamp'].apply(timestamp_to_date)

In [14]:
# solo por el primer día
# Reemplazar 'promedio' por 'average' en la columna 'Unnamed: 0'
combined_df['Unnamed: 0'] = combined_df['Unnamed: 0'].replace('promedio', 'average')

In [15]:
combined_df

,Unnamed: 0,Buying,Buying (final),Selling,Selling (final),Timestamp,Spread,Average dayly price,Fecha
0,bitmonedero,757.64,757.64,699.64,689.14,1.694638e+09,68.50,723.390,13 Sep 2023
1,kriptonmarket,746.75,758.47,703.25,692.37,1.694638e+09,66.10,725.420,13 Sep 2023
2,satoshitango,764.92,772.57,726.79,719.52,1.694638e+09,53.05,746.045,13 Sep 2023
3,saldo,732.99,732.99,691.27,691.27,1.694638e+09,41.72,712.130,13 Sep 2023
4,calypso,745.00,745.00,705.00,705.00,1.694638e+09,40.00,725.000,13 Sep 2023
...,...,...,...,...,...,...,...,...,...
633,bitsoalpha,938.55,944.18,922.66,917.13,1.697053e+09,27.05,930.655,11 Oct 2023
634,fiwind,950.00,950.00,928.00,928.00,1.697053e+09,22.00,939.000,11 Oct 2023
635,belo,945.00,945.00,928.00,928.00,1.697053e+09,17.00,936.500,11 Oct 2023
636,latamex,945.45,959.63,952.26,942.74,1.697053e+09,16.89,951.185,11 Oct 2023


In [16]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Supongamos que combined_df es el nombre de tu dataframe
# Asegúrate de importar pandas y numpy en tu script.

# Agrupa los datos por 'Fecha' y 'Unnamed: 0' y calcula el promedio
grouped_df = combined_df.groupby(['Fecha', 'Unnamed: 0'])['Average dayly price'].mean().reset_index()

# Convierte la columna 'Fecha' en un objeto de fecha
grouped_df['Fecha'] = pd.to_datetime(grouped_df['Fecha'])

# Ordena el DataFrame por la columna 'Fecha'
grouped_df = grouped_df.sort_values('Fecha')

# Create a meshgrid for the surface
mesh_x = grouped_df['Unnamed: 0'].unique()
mesh_y = grouped_df['Fecha'].unique()
mesh_x, mesh_y = np.meshgrid(mesh_x, mesh_y)
mesh_z = grouped_df.pivot(index='Fecha', columns='Unnamed: 0', values='Average dayly price').values

# Create a 3D surface plot
fig_3d = go.Figure(data=[go.Surface(
    x=mesh_x,
    y=mesh_y,
    z=mesh_z,
)])

# Configure the layout of the 3D surface plot
fig_3d.update_layout(
    title='Average Daily Prices by Exchange and Date',
    scene=dict(
        xaxis_title='Exchange',
        yaxis_title='Date',
        zaxis_title='Average Daily Price',
    ),
    margin=dict(l=50, r=50, b=150, t=50),  # Adjust margins as desired
)

# Show the interactive plot
fig_3d.show()


In [17]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define una paleta de colores personalizada
colores = {
    'average': "#AA322F",  # Bordó para 'average'
    'otro_nombre': 'rgb(128,128,128)'  # Gris para otros 'Unnamed: 0'
}

# Create subplots
fig3 = make_subplots(rows=1, cols=2, shared_xaxes=True)

# Inicializa una lista para las trazas del 'average'
trazas_average = []

# Iterate through each exchange in the DataFrame
for exchange in combined_df['Unnamed: 0'].unique():
    exchange_df = combined_df[combined_df['Unnamed: 0'] == exchange]

    # Obtén el color correspondiente de la paleta
    color = colores.get(exchange, '#dcdcdc')  # Gris por defecto

#######
    # Upper subplot: Average Daily Buying and Selling Prices for each exchange
    # Controla la visibilidad de la leyenda
    show_legend = True if exchange == 'average' else False    
    traza = go.Scatter(x=exchange_df['Fecha'], y=exchange_df['Average dayly price'], name=exchange, mode='lines', line=dict(width=2, color=color), showlegend=show_legend)
    
    if exchange == 'average':
        # Agrega la traza del 'average' a la lista
        trazas_average.append(traza)
    else:
        fig3.add_trace(traza, row=1, col=1)

####
    # Lower subplot: Spread Percentage for each exchange
    # Controla la visibilidad de la leyenda
    fig3.add_trace(go.Scatter(x=exchange_df['Fecha'], y=exchange_df['Spread'], name=exchange, mode='lines', line=dict(width=2, color=color), showlegend=False), row=1, col=2)

# Agrega primero la traza del 'average' al subplot izquierdo
for traza_average in trazas_average:
    fig3.add_trace(traza_average, row=1, col=1)


# Configure graph layout
fig3.update_layout(
    title=f'<b>USDT average bid/ask prices and spread in ARS for {current_date}<b>',
    yaxis=dict(
        title='ARS',
        title_standoff=25,
        title_font=dict(size=18),
        tickformat='~s'
    ),
    yaxis2=dict(
        title='Spread %',
        title_standoff=25,
        title_font=dict(size=18),
        tickformat='~s',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        font=dict(size=12),
        bgcolor='rgba(0,0,0,0)',
        # Oculta todas las leyendas en el layout
        itemsizing='constant'
    ),
    margin=dict(l=50, r=50, t=50, b=50),
    showlegend=True
)

# Show interactive graph
fig3.show()

# Sending to dashboard

In [18]:
pip install chart_studio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
import chart_studio
import chart_studio.plotly as py

In [20]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_3 = user_secrets.get_secret("CHARTSTUDIO_BCRA_API_KEY")

In [21]:
chart_studio.tools.set_credentials_file(username='crypto-financial-stability-monitor', 
                                        api_key=secret_value_3)
py.plot(fig, filename = 'arg-usdt-prices', auto_open=True)

'https://plotly.com/~crypto-financial-stability-monitor/634/'

In [22]:
py.plot(fig3, filename = 'arg-prices-usdt-times-series', auto_open=True)

'https://plotly.com/~crypto-financial-stability-monitor/743/'

In [23]:
py.plot(fig_3d, filename = 'arg-usdt-prices-3d', auto_open=True)

'https://plotly.com/~crypto-financial-stability-monitor/745/'